In [1]:
#importing required packages for this module
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime
from urllib.parse import urlparse,urlencode
import ipaddress
import time
import concurrent.futures
import pickle

In [197]:
top1kWebsites = pd.read_csv('./datasets/top1kwebsites.csv')
# top1kWebsites = top1kWebsites.sample(n = 50, random_state = 12).copy()
top1kWebsites.columns = ['Domain']
top1kWebsites.head()

,Domain
0,https://twitter.com
1,https://google.com
2,https://youtube.com
3,https://s.w.org
4,https://instagram.com


In [198]:
top1kWebsites.shape

(998, 1)

In [199]:
top1kWebsiteslist = top1kWebsites['Domain'].tolist()
# print(top1kWebsiteslist)

In [200]:
# Domain of the URL (Domain)
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
    domain = domain.replace("www.","")
  return domain

In [201]:
# Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip


In [202]:
# Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

In [203]:
# Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0
  else:
    length = 1
  return length

In [204]:
# Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [205]:
# Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [4]:
# Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpProtocol(url):
  protocol = urlparse(url).scheme
  # print(protocol)
  if 'https' in protocol:
    return 0
  else:
    return 1
  
# httpProtocol("https://www.googleapis.com")

https


0

In [207]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [208]:
#  Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [209]:
# Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            
    else:
        return 0            

In [210]:
# DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

In [211]:
# FIXEDF
def web_traffic(url):
    try:
        parsed_url = urlparse(url)


        domain_with_protocol = parsed_url.scheme + "://" + parsed_url.netloc

        domain_without_www = domain_with_protocol.replace('www.', '')
        
        if domain_without_www.startswith('http://'):
            domain_without_www = domain_without_www.replace('http://', 'https://')


        return 0 if domain_without_www in top1kWebsiteslist else 1
    except Exception as e:
        print(f"Error: {e}")
        return 1


In [212]:
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  print("creation date is", creation_date) #
  expiration_date = domain_name.expiration_date
  print("expiration date is", expiration_date) #
  print("type of creation date is", type(creation_date))
  print("type of expiration date is", type(expiration_date))
  
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
      print("stripped creation date is", creation_date)
      print("stripped expiration date is", expiration_date)
    except:
      return 1

  if type(creation_date) is list:
    creation_date = creation_date[0]
  if type(expiration_date) is list:
    expiration_date = expiration_date[0]
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [213]:
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if type(expiration_date) is list:
    expiration_date = expiration_date[0]
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 1 
    else:
      end = 0 
  return end

In [214]:
def iframe(response):
    if response == "":
        return 1  
    else:
        if re.search(r"<iframe\s+(?:(?!frameBorder).)*>", response.text, re.IGNORECASE):
            return 1  # Phishing: iframe tag found without frameBorder attribute
        else:
            return 0  # Legitimate: iframe tag found with frameBorder attribute or not found


In [215]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response):
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [216]:
def rightClick(response):
    if response == "":
        return 1  #Empty response
    else:
        if re.search(r"event\.button\s*===\s*2", response.text):
            return 1  # Phishing: Right click disabled
        else:
            return 0  # Legitimate: Right click not disabled


In [217]:
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [218]:
def featureExtraction(url):
    features = []
    # Address bar based features (10)
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpProtocol(url))
    features.append(tinyURL(url))
    features.append(prefixSuffix(url))

    print("Address bar features extracted.")

    # Domain based features (4)
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1

    features.append(dns)
    features.append(web_traffic(url))
    
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))

    print("Domain based features extracted.")

    # HTML & Javascript based features (4)
    try:
        response = requests.get(url, timeout=10)  # Timeout set to 10 seconds
    except Exception as e:
        print(f"Error fetching URL {url}: {str(e)}")
        response = ""

    features.append(iframe(response))
    features.append(mouseOver(response))
    features.append(rightClick(response))
    features.append(forwarding(response))

    print("HTML & Javascript based features extracted.")

    return features

In [219]:
# load model from file
loaded_model = pickle.load(open("./models/XGBoostClassifierMAXWithWebTrafficOLD.pickle.dat", "rb"))
loaded_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [220]:
#Extracting the feautres & storing them in a list
def test(input):
    url_features = []
    url = input 
    url_features.append(featureExtraction(url))
    feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards']

    urlFeatures = pd.DataFrame(url_features, columns= feature_names)
        #drop domain
    input_data = urlFeatures.drop(['Domain'], axis = 1).copy()
    predicted_classes = loaded_model.predict(input_data)
    return predicted_classes, urlFeatures

In [221]:
result, ftext = test("https://www.mha.gov.in/en/divisionofmha/administration-division")
print("Predicted result:", result)
ftext.head()

Address bar features extracted.
creation date is 2006-03-23 04:50:34
expiration date is 2024-03-23 04:50:34
type of creation date is <class 'datetime.datetime'>
type of expiration date is <class 'datetime.datetime'>
Domain based features extracted.


HTML & Javascript based features extracted.
Predicted result: [0]


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards
0,mha.gov.in,0,0,1,3,0,0,0,0,0,1,0,1,0,0,0,0
